#                        - Computational Neuroscience 2021-2022 Final Project -        

##   Project Name: Combinatorial Codes in Ventral Temporal Lobe for Visual Object Recognition



In [1]:
!pip install umap
!pip install pipreqs
!pip install lazypredict
!pip install nibabel
!pip install nilearn
!pip install -U kaleido


try:
    import sklearn
    print('Scikit-learn is available, version', sklearn.__version__)
    
except:
    !pip install scikit-learn
    
 
try:
    import cv2
    print('Open-CV is available, version', cv2.__version__)
    
except:
     !pip install opencv-python
    
   
try:
    import seaborn
    print('Seaborn is available, version', seaborn.__version__)
    
except:
     !pip install seaborn


Requirement already up-to-date: kaleido in d:\python\lib\site-packages (0.2.1)
Scikit-learn is available, version 0.23.1
Open-CV is available, version 4.5.1
Seaborn is available, version 0.11.0


In [4]:
from __future__ import print_function, division

# Basics:
import numpy as np,pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import os, random, time, sys, copy, math, pickle

# interactive mode
plt.ion()

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

# Dimension Reduction Algorithms:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import FastICA
from sklearn.decomposition import NMF
import umap

# Transformations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Metrics:
from sklearn.metrics import classification_report

# Train-Test Splitter:
from sklearn.model_selection import train_test_split

# For Classical ML algorithms:
from lazypredict.Supervised import LazyClassifier

# Utilies:
from tqdm import tqdm

# For distance measurements:
from scipy.spatial.distance import cdist

# Extras:
from abc import abstractmethod
from typing import Callable, Iterable, List, Tuple

# Set true for Google Colab:
COLAB = False

if COLAB:
    # To access Google Drive:
    from google.colab import drive
    drive.mount("/content/gdrive")

    
# For neuroimaging:
from nibabel.testing import data_path
from nilearn import plotting as nplt
from nilearn.input_data import NiftiMasker
from nilearn import datasets
from nilearn import plotting
from nilearn.image import mean_img
from nilearn.image import index_img
import nibabel as nib
from nilearn import image



print("NumPy Version: ", np.__version__)


root_dir = r'C:\Users\Administrator\Desktop\VOR'
os.chdir(root_dir)
image_results_dir = os.path.join(root_dir, 'images')
results_dir = os.path.join(root_dir, 'results')

print('Working Directory: \n ', root_dir)


# Creating requirements.txt file
!pip3 freeze > requirements.txt  

NumPy Version:  1.19.1
Working Directory: 
  C:\Users\Administrator\Desktop\VOR


In [5]:
from utils.timers import timeit
from utils.metrics import accuracy, confusion_matrix, visualize_confusion_matrix
from utils.savers import save, save_obj, load, load_obj
from utils.reproduce import random_seed
from dataset.fetch_data_matrix import fetch_from_haxby
from visualizer.plot2D import plot_2d
from visualizer.plot3D import plot_3d  

# Classical ML Algorithms

##   One Shot ML Classifiers

Applied Algorithms:

    * LinearSVC
    * SGDClassifier
    * MLPClassifier
    * Perceptron
    * LogisticRegression
    * LogisticRegressionCV
    * SVC
    * CalibratedClassifierCV
    * PassiveAggressiveClassifier
    * LabelPropagation
    * LabelSpreading
    * RandomForestClassifier
    * GradientBoostingClassifier
    * QuadraticDiscriminantAnalysis
    * RidgeClassifierCV
    * RidgeClassifier
    * AdaBoostClassifier
    * ExtraTreesClassifier
    * KNeighborsClassifier
    * BaggingClassifier
    * BernoulliNB
    * LinearDiscriminantAnalysis
    * GaussianNB
    * NuSVC
    * DecisionTreeClassifier
    * NearestCentroid
    * ExtraTreeClassifier
    * CheckingClassifier
    * DummyClassifier 

In [ ]:
%%time

# Loading:
fmri_imgs_mat, masks, categories = load('fMRI_data'), load('masked_data'), load('labels')


predictions_per_subject = list()


for subject_id, (mask, category) in enumerate(zip(masks, categories)):
    
    print(f'Subject id: {subject_id}')
  
    X_train, X_test, y_train, y_test = train_test_split(mask, category, test_size=0.3, random_state=42)
    
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)    
    
    models.to_csv(os.path.join(results_dir, f'Subject_{subject_id}_lazy_results.csv'))

    print(models)

## FREM : Ensembling of Regularized Models for Robust Decoding (SVC - L2)

FREM uses an implicit spatial regularization through fast clustering and aggregates a high number of estimators trained on various splits of the training set, thus returning a very robust decoder at a lower computational cost than other spatially regularized methods

---

FREM ensembling procedure yields an important improvement of decoding accuracy on this simple example compared to fitting only one model per fold and the clustering mechanism keeps its computational cost reasonable even on heavier examples. Here we ensembled several instances of l2-SVC, but FREMClassifier also works with ridge or logistic. 

In [ ]:
from nilearn.decoding import FREMClassifier
from nilearn.image import index_img
    
models_path = os.path.join(root_dir, 'models')
num_subjects = 6

for subject_id in range(num_subjects):
    
    print(f'Subject id: {subject_id}')

    behavioral = pd.read_csv(haxby_dataset.session_target[subject_id], sep=" ")

    conditions = behavioral['labels']
    condition_mask = conditions.isin([*stimuli2category])

    # Split data into train and test samples, using the chunks
    condition_mask_train = (condition_mask) & (behavioral['chunks'] <= 8)
    condition_mask_test = (condition_mask) & (behavioral['chunks'] > 8)
   
   
    filenames = haxby_dataset.func[subject_id]
    X_train = index_img(filenames, condition_mask_train)
    X_test = index_img(filenames, condition_mask_test)
    y_train = conditions[condition_mask_train].values
    y_test = conditions[condition_mask_test].values    
    
    masker = NiftiMasker(mask_img=haxby_dataset.mask_vt[subject_id],
                         smoothing_fwhm=4,
                         standardize=True,
                         memory='nilearn_cache',
                         memory_level=1)

    #masked = masker.fit_transform(fmri_niimgs)
    
    
    decoder = FREMClassifier(estimator='svc', cv=10, mask = masker)

    # Fit model on train data and predict on test data
    decoder.fit(X_train, y_train)

    y_pred = decoder.predict(X_test)
    
    report = pd.DataFrame(classification_report(y_test, y_pred, output_dict = True)).T      
    report.to_csv(os.path.join(results_dir, f'Subject_{subject_id}_FREM_results.csv')) 
    
    scores = pd.DataFrame(decoder.cv_scores_).T
    scores.to_csv(os.path.join(results_dir, f'Subject_{subject_id}_FREMCV_results.csv')) 
    
    save_obj(decoder, os.path.join(models_path, f'Subject_{subject_id}_FREM_model'))   

## FREM : Ensembling of Regularized Models for Robust Decoding (Logistic Regression - L2)

In [ ]:
from nilearn.decoding import FREMClassifier
from nilearn.image import index_img
from sklearn.model_selection import LeaveOneGroupOut
cv = LeaveOneGroupOut()  
models_path = os.path.join(root_dir, 'models')
num_subjects = 6

for subject_id in range(num_subjects):
    
    print(f'Subject id: {subject_id}')

    behavioral = pd.read_csv(haxby_dataset.session_target[subject_id], sep=" ")

    conditions = behavioral['labels']
    condition_mask = conditions.isin([*stimuli2category]) 
    
    filenames = haxby_dataset.func[subject_id]
    X_train = index_img(filenames, condition_mask)  
    y_train = conditions[condition_mask].values
    
    decoder = FREMClassifier(estimator='logistic_l2',
                             cv=10,
                             mask = NiftiMasker(mask_img=haxby_dataset.mask_vt[subject_id],
                                                 smoothing_fwhm=4,
                                                 standardize=True,
                                                 memory='nilearn_cache',
                                                 memory_level=1)
                            )

    # Fit model on train data and predict on test data:
    decoder.fit(X_train, y_train)
    
    # Saving:
    scores = pd.DataFrame(decoder.cv_scores_).T
    scores.to_csv(os.path.join(results_dir, f'Subject_{subject_id}_FREMLogisticRegressionCV_results.csv'))     
    save_obj(decoder, os.path.join(models_path, f'Subject_{subject_id}_FREMLogisticRegressionCV_model'))   

# ML Visualizations

In [ ]:
image_results_dir = os.path.join(root_dir,'images/results')

subject_id = 5
decoder = load_obj(os.path.join(models_path, f'Subject_{subject_id}_FREM_model'))

weight_img = decoder.coef_img_["face"]
filenames = haxby_dataset.func[subject_id]


plotting.plot_stat_map(weight_img,
                       bg_img = mean_img(filenames),
                       title=f"FREM: Accuracy Score for Face Stimuli: {np.mean(decoder.cv_scores_['face']).round(2)}",
                       cut_coords=(-52, -5),
                       display_mode="yz",
                       output_file= os.path.join(image_results_dir, 'FREM_face.png'),
                       )

plotting.show()

In [ ]:
subject_id = 5
decoder = load_obj(os.path.join(models_path, f'Subject_{subject_id}_FREM_model'))

weight_img = decoder.coef_img_["house"]
filenames = haxby_dataset.func[subject_id]

plotting.plot_stat_map(weight_img,
                       bg_img = mean_img(filenames),
                       title=f"FREM: Accuracy Score: {np.mean(decoder.cv_scores_['house']).round(2)}",
                       cut_coords=(-52, -5),
                       #output_file= os.path.join(image_results_dir, 'FREM_house.png'),
                       display_mode="yz")



plotting.show()

In [ ]:
subject_id = 0
decoder = load_obj(os.path.join(models_path, f'Subject_{subject_id}_FREM_model'))

weight_img = decoder.coef_img_["face"]


plotting.plot_stat_map(weight_img,
                       bg_img=haxby_dataset.anat[subject_id],
                       title='FREM (SVC-L2) Discriminating weights',
                       #output_file= os.path.join(image_results_dir, 'FREM (SVC-L2) Discriminating weights.png'),
                       )

plotting.show()

In [ ]:
subject_id = 0
decoder = load_obj(os.path.join(models_path, f'Subject_{subject_id}_FREM_model'))

weight_img = decoder.coef_img_["face"]


plotting.plot_stat_map(weight_img,
                       bg_img=haxby_dataset.anat[subject_id],
                       title='FREM (SVC-L2) Discriminating weights',
                       dim = -1,
                       #output_file= os.path.join(image_results_dir, 'FREM (SVC-L2) Discriminating weights anat.png')
                      )

plotting.show()